In [2]:
import os
from datetime import datetime
import time

import tweepy
from tweepy import OAuthHandler
import pandas as pd

auth = tweepy.OAuthHandler(os.environ.get('twitter_consumer_api_key'), os.environ.get('twitter_consumer_api_secret'))
auth.set_access_token(os.environ.get('twitter_access_api_key'), os.environ.get('twitter_access_api_secret'))

api = tweepy.API(auth, wait_on_rate_limit=True)

In [5]:
username = 'nba'
count = 50
df_user_tweets = pd.DataFrame()

In [27]:
df_test = []  
for tweet in tweepy.Cursor(api.search_tweets, 'nba', count = 50).items(250):
    df_test.append(tweet)

In [51]:
# result_type = 'popular'
df = pd.DataFrame()
try:
  for tweet in tweepy.Cursor(api.search_tweets, 'nba', count = 500, result_type = 'mixed').items(500):
      # print(status)
      df = df.append(
                            {'created_at' : tweet._json['created_at'],
                                        'tweet_id': tweet._json['id_str'],
                                        'user': tweet._json['user']['screen_name'],
                                        'user_id': tweet._json['user']['id'],
                                          'tweet': tweet._json['text'],
                      'likes': tweet._json['favorite_count'],
                          'retweets': tweet._json['retweet_count'],
                        'lang': tweet._json['lang'],
                            'scrape_ts': datetime.now(),
          'profile_img': tweet._json['user']['profile_image_url'],
          'url': f"https://twitter.com/twitter/statuses/{tweet._json['id']}",
                          }, ignore_index=True
      )
      
except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)

In [38]:
df_test[0]._json

{'created_at': 'Sat Jun 04 20:28:25 +0000 2022',
 'id': 1533183917301325826,
 'id_str': '1533183917301325826',
 'text': 'RT @joeylinn_: One year ago today, Kawhi Leonard dropped 45 PTS on 72% shooting to avoid elimination in Dallas.\n\n“I did not want to go home…',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'joeylinn_',
    'name': 'Joey Linn',
    'id': 2632100432,
    'id_str': '2632100432',
    'indices': [3, 13]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1115693823017897984,
  'id_str': '1115693823017897984',
  'name': '💫',
  'screen_name': '1629Ant',
  'location': 'Idk',
  'description': 'Sometimes you got

In [8]:
try:     
    # Creation of query method using appropriate parameters
    tweets =    tweepy.Cursor(api.user_timeline,id=username).items(count)
 
    
    # Pulling information from tweets iterable object and adding relevant tweet information in our data frame
    for tweet in tweets:
        df_user_tweets = df_user_tweets.append(
                          {'Created at' : tweet._json['created_at'],
                                       'User ID': tweet._json['id'],
                              'User Name': tweet.user._json['name'],
                                        'Text': tweet._json['text'],
                     'Description': tweet.user._json['description'],
                        'Likes': tweet.user._json['favorite_count'],
                      'Retweets': tweet.user._json['retweet_count'],
                           'Location': tweet.user._json['location'],
             'Followers Count': tweet.user._json['followers_count'],
                 'Friends Count': tweet.user._json['friends_count'],
               'Statuses Count': tweet.user._json['statuses_count'],
         'Profile Image Url': tweet.user._json['profile_image_url'],
                         }, ignore_index=True)
except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)

Unexpected parameter: id


failed on_status, 'favorite_count'


In [42]:
client = tweepy.Client(bearer_token=os.environ.get('bearer_token'))

In [44]:
df = client.search_recent_tweets('nba')

In [50]:
df

Response(data=[<Tweet id=1533186042714521600 text='@lorenzoggomes @carapanarana Assim ae Lorenzo https://t.co/03jxqzLx9Q'>, <Tweet id=1533186041279983616 text='RT @P_McCulloughMD: Tai et al, JAMA, NBA experience with mandated products a bust.  Staggering 608 confirmed infections in fully compliant.…'>, <Tweet id=1533186034657243136 text="RT @100ItsLevel: George Floyd wasn't real.  \n\nNobody died or went to jail. \n\nFloyd was played by crisis actor/NBA player Stephen Jackson. h…">, <Tweet id=1533186034636316674 text='NBB é muito bom, mas realmente existe um vão enorme entre ela e a NBA'>, <Tweet id=1533186033289920513 text='@BleacherReport @GabrielFC000 não sei se é pior a NBA season com 82 ou a MLB com 162!!!'>, <Tweet id=1533186027774529538 text='RT @PGN33: NBA players discovering CV woman bout to change the world 🤣😭'>, <Tweet id=1533186024943276038 text='RT @royal_bobby24: Felt every word nba youngboy said on Better Man https://t.co/0eudBY4bn3'>, <Tweet id=1533186022418223104 text=